Notebook for cleaning graphs from unrelated roads and apartment areas.

In [83]:
import matplotlib.pyplot as plt
%matplotlib inline
from imposm.parser import OSMParser
import networkx as nx
import pandas as pd
import numpy as np
import pickle as pkl
# import seaborn as sbn
import math
from tqdm import *

In [ ]:
G = nx.Graph()
node_lat = {}
node_lon = {}
graph_id = 0
with open('../bin/moscow_sample.ntw','r') as f:
    t = f.readline().split()
    graph_id = int(t[0])
    print(t[0])
    nodes = int(t[1])
    edges = int(t[2])
    for i in range(edges):
        info = f.readline().split()
        id1 = int(info[0])
        id2 = int(info[1])
        assert(id1 < nodes)
        assert(id2 < nodes)
        G.add_edge(id1, id2, weight=info[2], etype=info[3])
    for i in range(nodes):
        info = f.readline().split()
        node_lat[i] = info[0]
        node_lon[i] = info[1]
        if i not in G:
            G.add_node(i)
nx.set_node_attributes(G, 'lat', node_lat)
nx.set_node_attributes(G, 'lon', node_lon)

1501063804349715808


In [ ]:
to_delete = []
for edge in nx.edges_iter(G):
    if G[edge[0]][edge[1]]["etype"] in ["unrelated",'cycleway','pedestrian','footway','steps','path']:
        to_delete.append(edge)
for e in to_delete:
    G.remove_edge(e[0],e[1])
    
to_delete = []
for node in nx.nodes_iter(G):
    if G.degree(node) < 1:
        to_delete.append(node)
for n in to_delete:
    G.remove_node(n)

In [ ]:
len(G)

In [ ]:
graphs = list(nx.connected_component_subgraphs(G))
print(len(list(graphs)))
for g in graphs:
    if len(g) > 2000:
        print(len(g))

In [ ]:
for g in graphs:
    if len(g) > 2000:
        print(len(g))
        G = g
        break

In [68]:
len(list(nx.connected_component_subgraphs(G)))

1

In [73]:
#export with certain number of customers and readible format
new_node_map = {}
i = 0
for n in G.nodes(data=True):
    new_node_map[n[0]] = i
    i += 1

for sources in tqdm([2**x for x in range(7,17)]):    
    graph_id += 1
    with open('../data/aalborg/modified/' + str(graph_id) + ".ntw",'w') as f:
        f.write("%d %d %d %d\n" % (graph_id, len(G), nx.number_of_edges(G), sources)) 
        for e in G.edges(data=True):
            f.write("%d %d %s\n" % (new_node_map[e[0]], new_node_map[e[1]], e[2]['weight']))
        for i in np.random.choice(len(G), sources):
            f.write("%d\n" % i)
        i = 0
        for n in G.nodes(data=True):
            assert(new_node_map[n[0]] == i)
            i += 1
            f.write("%s %s\n" % (n[1]['lat'], n[1]['lon']))

100%|██████████| 10/10 [00:07<00:00,  1.29it/s]


In [82]:
#single file
new_node_map = {}
i = 0
for n in G.nodes(data=True):
    new_node_map[n[0]] = i
    i += 1

graph_id += 1
sources = 512
with open('../data/riga_512.ntw','w') as f:
    f.write("%d %d %d %d\n" % (graph_id, len(G), nx.number_of_edges(G), sources)) 
    for e in G.edges(data=True):
        f.write("%d %d %s\n" % (new_node_map[e[0]], new_node_map[e[1]], e[2]['weight']))
    for i in np.random.choice(len(G), sources):
        f.write("%d\n" % i)
    i = 0
    for n in G.nodes(data=True):
        assert(new_node_map[n[0]] == i)
        i += 1
        f.write("%s %s\n" % (n[1]['lat'], n[1]['lon']))

In [74]:
for n in G.nodes(data=True):
    print(n)
    break

(131072, {'lat': '10791.4', 'lon': '5700.07'})
